In [1]:
%config InlineBackend.figure_formats=['svg']
import numpy as np
import scipy
import scipy.sparse as sparse
import cvxpy as cp
import matplotlib.pyplot as plt
import only_prop_only_nuclear_norm as api
import time
import math

np.random.seed(0)

In [21]:
N_omega = 21
free_indices = 2*N_omega - 1 - 12
omega = np.linspace(-2, 2, N_omega)
delta_omega = np.abs(omega[1] - omega[0])
N_z = 6
z = np.linspace(0, 5*10**-3, N_z)
delta_z = np.abs(z[1] - z[0])
green_fs = api.get_green_f(omega,z)
projection = np.zeros((N_omega, N_omega))
projections = []
sdr_def_constr = []
sdr_cst = []
for i in range(N_omega):
    for j in range(N_omega):
        proj_copy = projection.copy()
        proj_copy[i, j] = 1
        projections.append(sparse.csc_matrix(proj_copy))
        sdr_def_constr.append(api.sdr_def_constr(N_omega, N_z, sparse.csc_matrix(proj_copy)))
        quad = api.diag_mat(N_omega, N_z, proj_copy)[0] + api.diag_mat(N_omega, N_z, proj_copy)[0].conj().T
        quad.resize(((2*N_z + 1)*N_omega,(2*N_z + 1)*N_omega))
        if i == j:
            sdr_cst.append(2.)
        else:
            sdr_cst.append(0.)

In [22]:
beta_vec = np.exp(-np.linspace(omega[0], omega[-1], 2*N_omega - 1)**2/2.1)
beta = scipy.linalg.hankel(beta_vec[:N_omega], beta_vec[N_omega - 1:])
new_beta = delta_omega*beta/(np.sqrt(np.trace(beta@beta)))
# Try for n = 0.5
beta_weight = 663.61#300.
delta_k = 1.j*np.diag(omega)
Q_plus = delta_k + beta_weight*new_beta
Q_minus = delta_k - beta_weight*new_beta
n = 0.25*np.trace((scipy.linalg.expm(Q_plus*z[-1]) - scipy.linalg.expm(Q_minus*z[-1])).conj().T@(scipy.linalg.expm(Q_plus*z[-1]) - scipy.linalg.expm(Q_minus*z[-1])))
W_plus = [(1/np.sqrt(n))*scipy.linalg.expm(Q_plus*z[i]) for i in range(1, N_z)]
W_minus = [(1/np.sqrt(n))*scipy.linalg.expm(Q_minus*z[i]) for i in range(1, N_z)]
X = np.vstack(W_plus + W_minus + [new_beta])
Y = np.vstack(W_plus + W_minus + [new_beta, np.eye(N_omega)])
full_rank = Y@Y.conj().T
n

(0.5000302986760162+0j)

In [102]:
# For N = 1
proj = projections[0]
real_plus_dyn, imag_plus_dyn, real_minus_dyn, imag_minus_dyn = api.get_dynamics_sdr(omega, z, proj, n, beta_weight)
approx_plus_0 = (1/np.sqrt(n))*(scipy.linalg.expm(delta_k*z[1])) + 0.5*delta_z*beta_weight*((1/np.sqrt(n))*scipy.linalg.expm(delta_k*z[1])@new_beta + new_beta@W_plus[0])
approx_minus_0 = (1/np.sqrt(n))*(scipy.linalg.expm(delta_k*z[1])) - 0.5*delta_z*beta_weight*((1/np.sqrt(n))*scipy.linalg.expm(delta_k*z[1])@new_beta + new_beta@W_minus[0])
np.trace(proj.conj().T@(approx_plus_0 - W_plus[0])), np.max(np.abs(np.real(approx_plus_0 - W_plus[0]))), np.trace(real_plus_dyn[0]@full_rank)

((7.451639446154346e-06+2.1183063816888545e-07j),
 1.5865910888201995e-05,
 (7.451639446154346e-06+2.168404344971009e-19j))

In [129]:
# For N = 1
proj = 1.j*projections[0]
real_plus_dyn, imag_plus_dyn, real_minus_dyn, imag_minus_dyn = api.get_dynamics_sdr(omega, z, proj, n, beta_weight)
approx_plus_1 = (1/np.sqrt(n))*(scipy.linalg.expm(delta_k*z[2])) + 0.5*delta_z*beta_weight*((1/np.sqrt(n))*scipy.linalg.expm(delta_k*z[2])@new_beta + new_beta@W_plus[1] + 2*scipy.linalg.expm(delta_k*z[1])@new_beta@W_plus[0])
approx_minus_1 = (1/np.sqrt(n))*(scipy.linalg.expm(delta_k*z[2])) - 0.5*delta_z*beta_weight*((1/np.sqrt(n))*scipy.linalg.expm(delta_k*z[2])@new_beta + new_beta@W_minus[1] + 2*scipy.linalg.expm(delta_k*z[1])@new_beta@W_plus[0])
np.trace(proj.conj().T@(approx_plus_1 - W_plus[1])),np.max(np.abs(np.real(approx_plus_1 - W_plus[1]))), np.max(np.abs(np.imag(approx_plus_1 - W_plus[1]))), np.trace(real_plus_dyn[1]@full_rank)

((4.3006826001400905e-07-1.599845714617132e-05j),
 3.3900966249555964e-05,
 7.990447812928134e-07,
 (4.3006826001357537e-07+0j))

In [112]:
# For N = 2
index = 2
green_f = api.get_green_f(omega, z[:index + 1])
green_f.reverse()
approx_plus = (1/np.sqrt(n))*scipy.linalg.expm(delta_k*z[index]) + 0.5*(delta_z*beta_weight/np.sqrt(n))*green_f[0]@new_beta + 0.5*delta_z*beta_weight*new_beta@W_plus[index - 1]
approx_minus = (1/np.sqrt(n))*scipy.linalg.expm(delta_k*z[index]) - 0.5*(delta_z*beta_weight/np.sqrt(n))*green_f[0]@new_beta - 0.5*delta_z*beta_weight*new_beta@W_minus[index - 1]
for i in range(index - 1):
    approx_plus += delta_z*beta_weight*(green_f[i + 1]@new_beta@W_plus[i])
    approx_minus -= delta_z*beta_weight*(green_f[i + 1]@new_beta@W_minus[i])

In [113]:
np.max(np.abs(np.real(approx_plus - W_plus[index - 1])))

3.390096624977801e-05